In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from datetime import date
import scipy.stats as st

In [2]:
df=pd.read_excel("/home/user/Desktop/env2/VAR/mvp_3/mixed_type.xlsx")

xls = pd.ExcelFile('/home/user/Desktop/env2/VAR/mvp_3/interest_rates.xlsx')
LIBOR_int=pd.read_excel(xls,"LIBOR")
LIBOR_int.index=LIBOR_int["Date"]

SOFR_int=pd.read_excel(xls,"SOFR")
SOFR_int.index=SOFR_int["Date"]

SONIA_int=pd.read_excel(xls,"SONIA")
SONIA_int.index=SONIA_int["Date"]

TIIE_int=pd.read_excel(xls,"TIIE")
TIIE_int.index=TIIE_int["Date"]

BSBY_int=pd.read_excel(xls,"BSBY")
BSBY_int.index=BSBY_int["Date"]
#print(TIIE_int)
df

,Loan ID,Segment,Region,Cost Centre,RM_ID,CAG_ID,GFCID,Facility Type,Facility ID,FRR,...,Start Date,Maturity Date,Comp_Freq,Duration (Period),Base Rate,RFR,Client Spread (LIBOR),Client Spread (RFR),Currency,Exposure
0,1,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Commercial Loan,F01,5,...,2019-12-31,2020-06-30,Monthly,6,LIBOR,SOFR,1.5,1.5,USD,5.046986e+05
1,2,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Commercial Loan,F02,5,...,2019-12-31,2020-12-31,Monthly,12,LIBOR,SOFR,1.5,1.5,USD,7.631238e+05
2,3,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Commercial Loan,F03,5,...,2019-12-31,2022-12-31,Monthly,36,LIBOR,SOFR,1.5,1.5,USD,9.628056e+05
3,4,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Equipment Loan,F04,6,...,2019-12-31,2024-12-31,Monthly,60,LIBOR,SOFR,1.5,1.5,USD,1.029663e+06
4,5,SEG-B,SAM,Uy,RM_02,G01_Rca,C02_JD,Equipment Loan,F05,6,...,2019-12-31,2029-12-31,Monthly,120,LIBOR,SOFR,1.5,1.5,USD,1.141394e+06
5,6,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Commercial Loan,F06,5,...,2019-04-02,2019-06-30,Monthly,3,LIBOR,SOFR,1.5,1.5,USD,3.365741e+06
6,7,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Commercial Loan,F07,5,...,2019-04-02,2019-09-30,Monthly,6,LIBOR,SOFR,1.5,1.5,USD,6.792669e+06
7,8,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Commercial Loan,F08,5,...,2019-04-02,2020-03-31,Monthly,12,LIBOR,SOFR,1.5,1.5,USD,8.624798e+06
8,9,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Equipment Loan,F09,6,...,2019-04-02,2019-09-30,Quarterly,2,LIBOR,SOFR,1.5,1.5,USD,6.796979e+06
9,10,SEG-A,NAM,US,RM_01,G01_RC,C01_JD,Equipment Loan,F10,6,...,2019-04-02,2020-03-31,Quarterly,4,LIBOR,SOFR,1.5,1.5,USD,8.641059e+06


In [3]:
base_rate_calc={"Monthly":"1M","Quarterly":"3M","Six Months":"6M","Annually":"12M"}
time_duration_calc= {"Monthly":1,"Quarterly":3,"Six Months":6,"Annually":12}
effective_col_calc={"Monthly":12,"Quarterly":4,"Six Months":2,"Annually":1}
mapping_dict={"LIBOR":LIBOR_int,"SOFR":SOFR_int,"SONIA":SONIA_int,"TIIE":TIIE_int,"BSBY":BSBY_int}
date_interval={"Monthly":30,"Quarterly":90,"Six Months":180,"Annually":360}
exposure_cal={"Monthly":3,"Quarterly":1,"Six Months":0.5,"Annually":0.25}

In [4]:
def numOfDays(date1, date2):
    "calculates number of days between two dates"
    return (date2-date1).days

def get_interest_rate(int_df,start_date,tenor):
    "returns interest rate for a particular date"
    interest_rate=int_df[int_df["Date"]==start_date][tenor].values
    return interest_rate

In [5]:
def fixed_rate_calculation(notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_fixed,RFR_dict):
    "calculates all interest and impact calculation for fixed rate portfolio"
    
    start_date, maturity_date=pd.to_datetime(start_date), pd.to_datetime(maturity_date)
    tenor= base_rate_calc[comp_freq]
    start_date_orig=start_date
    
    ####### Base Rate interest calculation #########
    int_df=mapping_dict[base_rate]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["Effective LIBOR"].append((1+(interest_rate[0]+spread_libor)/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["LIBOR Interest"].append(round(float(df_cal_fixed["Effective LIBOR"][-1])*notional,2))
    df_cal_fixed["LIBOR Total Interest"].append((notional*(1+df_cal_fixed["Effective LIBOR"][-1])**duration)-notional)
    no_of_days= numOfDays(start_date_orig,maturity_date)
    df_cal_fixed["YIELD P.A. % (LIBOR+CLIENT SPREAD)"].append(((((df_cal_fixed["LIBOR Total Interest"][-1]/notional)+1)**(365/no_of_days))-1)*100)
    
    ############ RFR Interest Calculation ##########
    int_df=mapping_dict[rfr]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["Effective RFR"].append((1+(interest_rate[0]+spread_rfr)/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["RFR Interest"].append(round(float(df_cal_fixed["Effective RFR"][-1])*notional,2))
    df_cal_fixed["RFR Total Interest"].append((notional*(1+df_cal_fixed["Effective RFR"][-1])**duration)-notional)
    no_of_days= numOfDays(start_date,maturity_date)
    df_cal_fixed["YIELD P.A. % (RFR+CLIENT SPREAD)"].append(((((df_cal_fixed["RFR Total Interest"][-1]/notional)+1)**(365/no_of_days))-1)*100)
    
    ########## RFR Calculated spread and interest calculation ########
    df_cal_fixed["NET NEUTRALITY YIELD PREMIUM"].append(df_cal_fixed["YIELD P.A. % (RFR+CLIENT SPREAD)"][-1]-df_cal_fixed["YIELD P.A. % (LIBOR+CLIENT SPREAD)"][-1])
    df_cal_fixed["NET NEUTRALITY YIELD PREMIUM (BP)"].append(((df_cal_fixed["NET NEUTRALITY YIELD PREMIUM"][-1]/100)*notional)/1000)
    df_cal_fixed["Impact (BP)"].append(((df_cal_fixed["RFR Total Interest"][-1]-df_cal_fixed["LIBOR Total Interest"][-1])/notional)*10000)
#     emi=(notional*df_cal_fixed["Effective LIBOR"][-1]*(1+df_cal_fixed["Effective LIBOR"][-1])**duration)/(((1+df_cal_fixed["Effective LIBOR"][-1])**duration)-1)
#     df_cal_fixed["EXPOSURE"].append(emi*(duration-exposure_cal[comp_freq]))
    df_cal_fixed["RFR Spread Cal"].append(((((df_cal_fixed["Effective LIBOR"][-1]+1)**effective_col_calc[comp_freq])-1)*100)-interest_rate[0])
    df_cal_fixed["Effective RFR Cal"].append((1+(interest_rate[0]+df_cal_fixed['RFR Spread Cal'][-1])/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["Interest Calc"].append(round(float(df_cal_fixed["Effective RFR Cal"][-1])*notional,2))
    df_cal_fixed["Total Interest Calc"].append((notional*(1+df_cal_fixed["Effective RFR Cal"][-1])**duration)-notional)
    
    
    RFR_dict[rfr].append(df_cal_fixed["RFR Spread Cal"][-1])
    return df_cal_fixed,RFR_dict 

def adjusted_RFR_interest_calculation(df_cal_fixed,RFR_dict,notional,rfr,start_date,duration,comp_freq,i):
    
    tenor= base_rate_calc[comp_freq]
    int_df=mapping_dict[rfr]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["RFR Spread Adj"].append(round(np.mean(RFR_dict[rfr]),4))
    df_cal_fixed["Effective RFR Adj"].append((1+(interest_rate[0]+df_cal_fixed['RFR Spread Adj'][-1])/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["Interest Adj"].append(round(float(df_cal_fixed["Effective RFR Adj"][-1])*notional,2))
    df_cal_fixed["Total Interest Adj"].append((notional*(1+df_cal_fixed["Effective RFR Adj"][-1])**duration)-notional)
    df_cal_fixed["Impact"].append(df_cal_fixed["Total Interest Adj"][-1] - df_cal_fixed["Total Interest Calc"][i])
    df_cal_fixed["EMI RFR"].append((notional*df_cal_fixed["Effective LIBOR"][i]*(1+df_cal_fixed["Effective LIBOR"][i])**duration)/(((1+df_cal_fixed["Effective LIBOR"][i])**duration)-1))
    return df_cal_fixed                                   

In [6]:
inter_dict_base={}
inter_dict_RFR={}
inter_dict_RFR_adj={}

def floating_rate_calculation(loan_id,notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_floating):
    ######## calculation of interest for floating rate portfolio #########
    
    inter_dict_base[loan_id]=[]
    inter_dict_RFR[loan_id]=[]
    inter_dict_RFR_adj[loan_id]=[]
    interest_monthly=[]
    start_date_orig=start_date
    for j in range(0,duration):
        ####Calculation of interest with Base Rate######

        start_date= start_date_orig+timedelta(days=int(j)*date_interval[comp_freq])
        int_df=mapping_dict[base_rate]
        tenor= base_rate_calc[comp_freq]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_libor)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_base[loan_id].append(notional*effective_rate)
        
        ####Calculation of interest with RFR Rate######
        int_df=mapping_dict[rfr]
        interest_rate=get_interest_rate(int_df,start_date,tenor)

        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_rfr)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_RFR[loan_id].append(notional*effective_rate)
    
    df_cal_floating["LIBOR Total Interest"].append(np.sum(inter_dict_base[loan_id]))
    df_cal_floating["RFR Total Interest"].append(np.sum(inter_dict_RFR[loan_id]))
    no_of_days=numOfDays(start_date_orig,maturity_date)
    df_cal_floating["YIELD P.A. % (LIBOR+CLIENT SPREAD)"].append((((df_cal_floating["LIBOR Total Interest"][-1]/notional)+1)**(365/no_of_days)-1)*100)
    df_cal_floating["YIELD P.A. % (RFR+CLIENT SPREAD)"].append((((df_cal_floating["RFR Total Interest"][-1]/notional)+1)**(365/no_of_days)-1)*100)
    df_cal_floating["NET NEUTRALITY YIELD PREMIUM"].append(df_cal_floating["YIELD P.A. % (RFR+CLIENT SPREAD)"][-1]-df_cal_floating["YIELD P.A. % (LIBOR+CLIENT SPREAD)"][-1])
    df_cal_floating["NET NEUTRALITY YIELD PREMIUM (BP)"].append(((df_cal_floating["NET NEUTRALITY YIELD PREMIUM"][-1]/100)*notional)/1000)
#     emi=((notional+df_cal_floating["LIBOR Total Interest"])/duration)
#     df_cal_floating["EXPOSURE"]=(emi*(duration-exposure_cal[comp_freq]))
    ###weighted AVG Calculation###

    RFR_Weighted_AVG=(((((df_cal_floating["RFR Total Interest"][-1]/notional)/duration)+1)**effective_col_calc[comp_freq])-1)*100
    df_cal_floating["RFR Spread Adj"].append(((((1+((df_cal_floating["LIBOR Total Interest"][-1]/duration)/notional))**effective_col_calc[comp_freq])-1)*100)-RFR_Weighted_AVG)
    Adjusted_spred=df_cal_floating["RFR Spread Adj"][-1]
    
    ####Calculation of interest with RFR adjusted Rate######
    for j in range(0,duration):
        start_date= start_date_orig+timedelta(days=int(j)*date_interval[comp_freq])
        int_df=mapping_dict[rfr]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_rfr+Adjusted_spred)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_RFR_adj[loan_id].append(notional*effective_rate)
    
    df_cal_floating["Total Interest Adj"].append(np.sum(inter_dict_RFR_adj[loan_id]))
    return df_cal_floating

In [7]:
df_cal_fixed={"Effective LIBOR":[],"LIBOR Interest":[],"LIBOR Total Interest":[],"Effective RFR":[],"RFR Interest":[],"RFR Total Interest":[],"YIELD P.A. % (LIBOR+CLIENT SPREAD)":[],"YIELD P.A. % (RFR+CLIENT SPREAD)":[],"NET NEUTRALITY YIELD PREMIUM":[],"NET NEUTRALITY YIELD PREMIUM (BP)":[],"Impact (BP)":[],"RFR Spread Cal":[],"Effective RFR Cal":[],"Interest Calc":[],"Total Interest Calc":[],"RFR Spread Adj":[],"Effective RFR Adj":[],"Interest Adj":[],"Total Interest Adj":[],"Impact":[],"EMI RFR":[]}
df_cal_floating={"LIBOR Total Interest":[],"RFR Total Interest":[],"YIELD P.A. % (LIBOR+CLIENT SPREAD)":[],"YIELD P.A. % (RFR+CLIENT SPREAD)":[],"NET NEUTRALITY YIELD PREMIUM":[],"NET NEUTRALITY YIELD PREMIUM (BP)":[],"RFR Spread Adj":[],"Total Interest Adj":[]}

rfr=list(df["RFR"].unique())
RFR_dict={}
for item in rfr:
    RFR_dict[item]=[]
for i in range(len(df)):
    loan_id,notional,start_date,maturity_date=df.loc[i,"Loan ID"],df.loc[i,"Notional Amt"],df.loc[i,"Start Date"],df.loc[i,"Maturity Date"]
    comp_freq,duration,nature=df.loc[i,"Comp_Freq"],df.loc[i,"Duration (Period)"],df.loc[i,"Loan Nature"]
    base_rate,rfr,spread_libor,spread_rfr,currency=df.loc[i,"Base Rate"],df.loc[i,"RFR"],df.loc[i,"Client Spread (LIBOR)"],df.loc[i,"Client Spread (RFR)"],df.loc[i,"Currency"]
    if nature=="Fixed":
        df_cal_fixed,RFR_dict=fixed_rate_calculation(notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_fixed,RFR_dict)
    if nature=="Floating":
        df_cal_floating = floating_rate_calculation(loan_id,notional,start_date,maturity_date,comp_freq,duration,base_rate,rfr,spread_libor,spread_rfr,currency,df_cal_floating)
for i in range(len(df)):
    notional,start_date,rfr,duration,comp_freq,nature= df.loc[i,"Notional Amt"],df.loc[i,"Start Date"],df.loc[i,"RFR"],df.loc[i,"Duration (Period)"],df.loc[i,"Comp_Freq"],df.loc[i,"Loan Nature"]
    if nature=="Fixed":
        df_cal_fixed=adjusted_RFR_interest_calculation(df_cal_fixed,RFR_dict,notional,rfr,start_date,duration,comp_freq,i)
int_df_fixed=pd.DataFrame(df_cal_fixed)
int_df_floating= pd.DataFrame(df_cal_floating)
int_df_fixed

,Effective LIBOR,LIBOR Interest,LIBOR Total Interest,Effective RFR,RFR Interest,RFR Total Interest,YIELD P.A. % (LIBOR+CLIENT SPREAD),YIELD P.A. % (RFR+CLIENT SPREAD),NET NEUTRALITY YIELD PREMIUM,NET NEUTRALITY YIELD PREMIUM (BP),...,RFR Spread Cal,Effective RFR Cal,Interest Calc,Total Interest Calc,RFR Spread Adj,Effective RFR Adj,Interest Adj,Total Interest Adj,Impact,EMI RFR
0,0.002679,2678.92,16181.578262,0.002505,2504.6,15122.012371,3.271608,3.055768,-0.215840,-2.158397,...,1.71523,0.002679,2678.92,16181.578262,1.7152,0.002679,2678.9,16181.430651,-0.147611,168232.856244
1,0.002679,2678.92,32625.000000,0.002505,2504.6,30472.700000,3.253443,3.038819,-0.214624,-2.146237,...,1.71523,0.002679,2678.92,32625.000000,1.7152,0.002679,2678.9,32624.700000,-0.300000,84791.534263
2,0.002679,2678.92,101102.897619,0.002505,2504.6,94232.152842,3.259475,3.044448,-0.215028,-2.150275,...,1.71523,0.002679,2678.92,101102.897619,1.7152,0.002679,2678.9,101101.937936,-0.959683,29175.928210
3,0.002679,2678.92,174121.865290,0.002505,2504.6,161936.657140,3.258871,3.043884,-0.214987,-2.149871,...,1.71523,0.002679,2678.92,174121.865290,1.7152,0.002679,2678.9,174120.159752,-1.705539,18064.262843
4,0.002679,2678.92,378562.154552,0.002505,2504.6,350096.795206,3.259777,3.044730,-0.215048,-2.150478,...,1.71523,0.002679,2678.92,378562.154552,1.7152,0.002679,2678.9,378558.149535,-4.005017,9755.500059


In [8]:
int_df_floating

,LIBOR Total Interest,RFR Total Interest,YIELD P.A. % (LIBOR+CLIENT SPREAD),YIELD P.A. % (RFR+CLIENT SPREAD),NET NEUTRALITY YIELD PREMIUM,NET NEUTRALITY YIELD PREMIUM (BP),RFR Spread Adj,Total Interest Adj
0,97223.245858,96844.895589,4.047764,4.031776,-0.015988,-1.598831,0.015681,97223.245869
1,189003.091483,191022.444303,3.848007,3.889516,0.041508,4.150847,-0.041816,189003.090408
2,349757.772206,350232.352944,3.507353,3.512112,0.004759,0.475930,-0.004900,349757.769268
3,195468.878000,194076.989281,3.980943,3.952319,-0.028624,-2.862432,0.028659,195468.878000
4,369270.622054,366014.913480,3.703037,3.670387,-0.032650,-3.264977,0.033463,369270.633091
5,573090.776544,553403.600398,2.829466,2.733555,-0.095911,-9.591056,0.100530,573091.494885


In [9]:
df_fixed=df[df["Loan Nature"]=="Fixed"]
df_fixed.index=pd.RangeIndex(len(df_fixed))
#print(df_fixed)
df_agg_yield_cal_fi=int_df_fixed.loc[:, :'NET NEUTRALITY YIELD PREMIUM (BP)']
df_agg_yield_cal_fi=pd.concat([df_fixed,df_agg_yield_cal_fi],axis=1)

df_floating=df[df["Loan Nature"]=="Floating"]
df_floating.index=pd.RangeIndex(len(df_floating))
#print(df_fixed)
df_agg_yield_cal_fl=int_df_floating.loc[:, :'NET NEUTRALITY YIELD PREMIUM (BP)']
df_agg_yield_cal_fl=pd.concat([df_floating,df_agg_yield_cal_fl],axis=1)
#print(df_agg_yield_cal_fl)

def yield_aggregation(df_agg_yield_cal):
    
    Total_exposure=np.sum(df_agg_yield_cal["Exposure"])
    #print(Total_exposure)
    for i in range(len(df_agg_yield_cal)):
        df_agg_yield_cal.loc[i,"Weights"]=df_agg_yield_cal.loc[i,"Exposure"]/Total_exposure
        df_agg_yield_cal.loc[i,"Yield WA LIBOR"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"YIELD P.A. % (LIBOR+CLIENT SPREAD)"]
        df_agg_yield_cal.loc[i,"Yield WA RFR"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"YIELD P.A. % (RFR+CLIENT SPREAD)"]
        df_agg_yield_cal.loc[i,"Net Neutrality Yield"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"NET NEUTRALITY YIELD PREMIUM"]
        df_agg_yield_cal.loc[i,"Net Neutrality Yield (BP)"]=df_agg_yield_cal.loc[i,"Weights"]*df_agg_yield_cal.loc[i,"NET NEUTRALITY YIELD PREMIUM (BP)"]
    #print(df_agg_yield_cal)

    Yield_stat={"Total Exposure":Total_exposure,"WA Yield (LIBOR)":np.sum(df_agg_yield_cal["Yield WA LIBOR"]),"WA Yield (RFR)":np.sum(df_agg_yield_cal["Yield WA RFR"]),"WA Yield Difference":np.sum(df_agg_yield_cal["Net Neutrality Yield"]),"WA Yield Difference (BP)":np.sum(df_agg_yield_cal["Net Neutrality Yield (BP)"])}
    yield_df=pd.DataFrame(Yield_stat,index=[0])
    #print(yield_df)
    return df_agg_yield_cal,yield_df



# ###Yield agrregation segment###

Segment= "SEG-A"
df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["Segment"]==Segment]
df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["Segment"]==Segment]
df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))

# ###Yield agrregation region###

# Region= "NAM"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["Region"]==Region]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["Region"]==Region]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))

# ###Yield agrregation cost centre###

# Cost_centre= "US"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["Cost Centre"]==Cost_centre]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["Cost Centre"]==Cost_centre]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))

# ###Yield agrregation RM_ID###

# RM_ID= "RM_01"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["RM_ID"]==RM_ID]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["RM_ID"]==RM_ID]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))


# ###Yield agrregation CAG_ID###

# CAG_ID= "G01_Rca"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["CAG_ID"]==CAG_ID]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["CAG_ID"]==CAG_ID]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))

# ###Yield agrregation GFCID###

# GFC_ID= "C02_JD"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["GFCID"]==GFC_ID]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["GFCID"]==GFC_ID]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))

# ###Yield agrregation Facility Type###

# Type= "Equipment Loan"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["Facility Type"]==Type]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["Facility Type"]==Type]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))

# ###Yield agrregation region###

# Region= "NAM"
# df_agg_yield_cal_fi=df_agg_yield_cal_fi[df_agg_yield_cal_fi["Region"]==Region]
# df_agg_yield_cal_fi.index=pd.RangeIndex(len(df_agg_yield_cal_fi.index))
# df_agg_yield_cal_fl=df_agg_yield_cal_fl[df_agg_yield_cal_fl["Region"]==Region]
# df_agg_yield_cal_fl.index=pd.RangeIndex(len(df_agg_yield_cal_fl.index))








In [10]:
try:
    df_agg_yield_cal_fi,fi_summary_df=yield_aggregation(df_agg_yield_cal_fi)
    print(df_agg_yield_cal_fi)
    print(fi_summary_df)
except:
    pass

   Loan ID Segment Region Cost Centre  RM_ID   CAG_ID   GFCID   Facility Type  \
0        5   SEG-B    SAM          Uy  RM_02  G01_Rca  C02_JD  Equipment Loan   

  Facility ID  FRR  ...  RFR Total Interest  \
0         F05    6  ...       350096.795206   

  YIELD P.A. % (LIBOR+CLIENT SPREAD) YIELD P.A. % (RFR+CLIENT SPREAD)  \
0                           3.259777                          3.04473   

  NET NEUTRALITY YIELD PREMIUM NET NEUTRALITY YIELD PREMIUM (BP)  Weights  \
0                    -0.215048                         -2.150478      1.0   

  Yield WA LIBOR Yield WA RFR  Net Neutrality Yield  Net Neutrality Yield (BP)  
0       3.259777      3.04473             -0.215048                  -2.150478  

[1 rows x 37 columns]
   Total Exposure  WA Yield (LIBOR)  WA Yield (RFR)  WA Yield Difference  \
0    1.141394e+06          3.259777         3.04473            -0.215048   

   WA Yield Difference (BP)  
0                 -2.150478  


In [11]:
try:
    df_agg_yield_cal_fl,fl_summary_df=yield_aggregation(df_agg_yield_cal_fl)
    print(df_agg_yield_cal_fl)
    print(fl_summary_df)
except:
    pass